In [51]:
import torch
import open_clip
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from glob import glob
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

# === 1. Load CLIP Model ===
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')

def load_clip():
    tokenizer = open_clip.get_tokenizer('ViT-B-32')
    return model, preprocess

# === 2. Pupil Cropping ===
def crop_to_pupil(image_path, output_size=(784, 784)):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.medianBlur(gray, 5)

    circles = cv2.HoughCircles(gray_blur, cv2.HOUGH_GRADIENT, dp=1.5, minDist=30,
                                param1=50, param2=30, minRadius=20, maxRadius=150)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        x, y, r = circles[0][0]
        pad = int(r * 1.5)
        x1, y1 = max(0, x - pad), max(0, y - pad)
        x2, y2 = min(image.shape[1], x + pad), min(image.shape[0], y + pad)
        cropped = image[y1:y2, x1:x2]
    else:
        print(f"⚠️ Pupil not detected in {image_path}, using full image.")
        cropped = image

    resized = cv2.resize(cropped, output_size)
    return resized

# === 3. Save Cropped Images ===
def preprocess_folder(input_folder, output_folder, size=(784, 784)):
    os.makedirs(output_folder, exist_ok=True)
    for fname in tqdm(os.listdir(input_folder)):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            path = os.path.join(input_folder, fname)
            cropped = crop_to_pupil(path, output_size=size)
            save_path = os.path.join(output_folder, fname)
            cv2.imwrite(save_path, cropped)

# === 4. Augmentation (optional) ===
augmentation_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.2)
])

def augment_image(img_pil, n_augmentations=2):
    augmented = [augmentation_transforms(img_pil) for _ in range(n_augmentations)]

    # Grayscale version
    grayscale = transforms.Grayscale()(img_pil)
    augmented.append(grayscale.convert("RGB"))  # Ensure 3 channels

    # Edge-detected (Canny via OpenCV)
    np_img = np.array(img_pil)
    gray = cv2.cvtColor(np_img, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    edge_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    edge_pil = Image.fromarray(edge_rgb)
    augmented.append(edge_pil)

    return augmented

# === 5. Embedding Extraction ===
def get_embeddings(image_path, preprocess, augment=False, n_aug=3):
    image = Image.open(image_path).convert("RGB")
    images = [image]
    if augment:
        images += augment_image(image, n_augmentations=n_aug)

    embeddings = []
    for img in images:
        tensor = preprocess(img).unsqueeze(0)
        with torch.no_grad():
            feat = model.encode_image(tensor)
            feat = feat / feat.norm(dim=-1, keepdim=True)
        embeddings.append(feat.numpy())

    return np.mean(embeddings, axis=0)

# === 6. Build DataFrame from Folder ===
def build_dataframe(folder, label, preprocess, augment=True, n_aug=3):
    df = pd.DataFrame(columns=range(512))
    idx = 0
    for image_path in tqdm(glob(f"{folder}/*.png")):
        image = Image.open(image_path).convert("RGB")
        # Original
        tensor = preprocess(image).unsqueeze(0)
        with torch.no_grad():
            feat = model.encode_image(tensor)
            feat = feat / feat.norm(dim=-1, keepdim=True)
#         print(feat.numpy(),len(feat.numpy()[0]))
        df.loc[idx, list(range(512))] = list(feat.numpy()[0])
        df.loc[idx, 'category'] = label
        idx += 1

        # Augmented
        if augment:
            augmented_imgs = augment_image(image, n_augmentations=n_aug)
            for img in augmented_imgs:
                tensor = preprocess(img).unsqueeze(0)
                with torch.no_grad():
                    feat = model.encode_image(tensor)
                    feat = feat / feat.norm(dim=-1, keepdim=True)
                df.loc[idx, list(range(512))] = list(feat.numpy()[0])
                df.loc[idx, 'category'] = label
                idx += 1

    return df


# === 7. Train Model ===
def train_classifier(X, y):
    model = LGBMClassifier(random_state=42,**{'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200})
    model.fit(X, y)
    return model

# === 8. Evaluate Model ===
def evaluate_model(model, X_val, y_val):
    y_pred = model.predict(X_val)
    print(confusion_matrix(y_val, y_pred))
    print(classification_report(y_val, y_pred))


    

In [44]:
model, preprocess = load_clip()


In [45]:
import os

# Run once to preprocess images
preprocess_folder("processed_eye_images//train/normal", "processed_aug/train/normal")
preprocess_folder("processed_eye_images/train/cataract", "processed_aug/train/cataract")


100%|█| 246/246 [00:05<00:00, 4
100%|█| 245/245 [00:05<00:00, 4


In [46]:
df

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,category
0,-0.008209,-0.025388,0.170292,-0.076664,-0.011962,-0.046452,0.011933,0.052535,-0.040558,0.072361,...,0.015106,-0.014476,-0.018878,-0.042237,0.009638,0.021294,0.028043,-0.001386,-0.018250,0.0
1,0.029179,0.133555,-0.067828,-0.077476,0.001874,-0.029483,0.039485,-0.019857,-0.033151,0.054449,...,-0.007106,0.022177,0.007312,-0.036768,0.027888,0.012523,0.011096,-0.014833,0.007427,0.0
2,0.033629,0.070846,0.069969,-0.076290,-0.015656,-0.042773,0.014903,0.011056,-0.020503,0.038118,...,0.039820,-0.005678,-0.026400,-0.025397,-0.030248,0.015482,-0.009309,-0.024477,-0.004683,0.0
3,0.020165,-0.006758,-0.010106,-0.073266,0.015090,-0.055530,0.015667,-0.010074,-0.026962,0.074223,...,0.014216,-0.000033,0.028111,-0.046412,-0.042317,0.044431,0.005613,0.015509,-0.014756,1.0
4,0.008486,0.018246,-0.070254,-0.082442,-0.006956,-0.037441,0.034377,0.007716,-0.003671,0.010978,...,0.014514,0.008325,0.036520,-0.056718,-0.012834,0.018885,-0.021029,0.011422,-0.005907,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,0.019468,0.006743,0.084389,-0.089938,-0.001608,-0.014791,0.010211,0.022423,-0.027107,0.044697,...,0.078755,0.005462,-0.032743,-0.016124,-0.041180,0.010931,-0.004320,0.004352,0.012522,0.0
487,0.045920,0.040774,0.080929,-0.087540,-0.007057,-0.006279,0.027199,0.011280,-0.017314,0.064901,...,0.066825,0.015700,-0.030704,-0.024800,-0.052801,0.002484,-0.015694,0.006363,0.043656,0.0
488,0.009453,-0.051162,0.103528,-0.092911,0.020825,-0.039286,0.013243,0.008333,-0.024395,0.046551,...,0.059349,0.014583,-0.007841,-0.051091,-0.025609,0.025573,-0.019856,0.025146,0.012634,1.0
489,0.020057,0.124875,-0.043288,-0.019162,0.034511,-0.058726,0.000202,-0.003096,-0.040012,0.063139,...,0.025831,0.002300,-0.029976,-0.039640,-0.006650,0.007998,-0.017334,-0.002568,0.037007,1.0


In [47]:

df_normal = build_dataframe("processed_aug/train/normal", 0, preprocess)
df_cataract = build_dataframe("processed_aug/train/cataract", 1, preprocess)
df = pd.concat([df_normal, df_cataract]).astype(float).sample(frac=1).reset_index(drop=True)

X = df.iloc[:, :512]
y = df['category']



100%|█| 246/246 [00:58<00:00,  
100%|█| 245/245 [00:58<00:00,  


[LightGBM] [Info] Number of positive: 1176, number of negative: 1180
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004185 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 130560
[LightGBM] [Info] Number of data points in the train set: 2356, number of used features: 512
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499151 -> initscore=-0.003396
[LightGBM] [Info] Start training from score -0.003396
[[280  16]
 [ 19 275]]
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.94       296
         1.0       0.95      0.94      0.94       294

    accuracy                           0.94       590
   macro avg       0.94      0.94      0.94       590
weighted avg       0.94      0.94      0.94       590



In [52]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)
clf = train_classifier(X_train, y_train)
evaluate_model(clf, X_val, y_val)

[[278  18]
 [ 17 277]]
              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94       296
         1.0       0.94      0.94      0.94       294

    accuracy                           0.94       590
   macro avg       0.94      0.94      0.94       590
weighted avg       0.94      0.94      0.94       590



In [48]:
X.shape

(2946, 512)

In [49]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
}

grid = GridSearchCV(LGBMClassifier(verbose=-1), param_grid, cv=3)
grid.fit(X_train, y_train)

print("Best model:", grid.best_estimator_)

Best model: LGBMClassifier(max_depth=7, n_estimators=200, verbose=-1)


In [50]:
grid.best_params_

{'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200}

In [53]:
import os
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm
import pandas as pd

def crop_to_pupil(image_path, output_size=(784, 784)):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.medianBlur(gray, 5)

    circles = cv2.HoughCircles(gray_blur, cv2.HOUGH_GRADIENT, dp=1.5, minDist=30,
                                param1=50, param2=30, minRadius=20, maxRadius=150)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        x, y, r = circles[0][0]
        pad = int(r * 1.5)
        x1, y1 = max(0, x - pad), max(0, y - pad)
        x2, y2 = min(image.shape[1], x + pad), min(image.shape[0], y + pad)
        cropped = image[y1:y2, x1:x2]
    else:
        print(f"⚠️ Pupil not detected in {image_path}, using full image.")
        cropped = image

    resized = cv2.resize(cropped, output_size)
    return resized

def infer_on_folder(folder_path, clf, preprocess, model):
    results = []
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for image_path in tqdm(image_paths, desc="🔍 Running Inference"):
        cropped = crop_to_pupil(image_path)
        image_pil = Image.fromarray(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
        image_tensor = preprocess(image_pil).unsqueeze(0)

        with torch.no_grad():
            embedding = model.encode_image(image_tensor)
            embedding = embedding / embedding.norm(dim=-1, keepdim=True)

        pred = clf.predict(embedding.numpy())[0]
        prob = clf.predict_proba(embedding.numpy())[0]

        results.append({
            'image_path': image_path,
            'prediction': 'cataract' if pred == 1 else 'normal',
            'prob_cataract': round(prob[1], 4),
            'prob_normal': round(prob[0], 4),
        })

    return pd.DataFrame(results)


In [54]:
model, preprocess = load_clip()

# Run inference
df_results = infer_on_folder("processed_images/test/cataract/", clf, preprocess, model)

# Show results
print(df_results.head())


🔍 Running Inference:  62%|▌| 3libpng warning: iCCP: known incorrect sRGB profile
🔍 Running Inference: 100%|█| 6

                                     image_path prediction  prob_cataract  \
0  processed_images/test/cataract/image_279.png   cataract         1.0000   
1  processed_images/test/cataract/image_251.png   cataract         0.9936   
2  processed_images/test/cataract/image_292.png   cataract         1.0000   
3  processed_images/test/cataract/image_286.png   cataract         1.0000   
4  processed_images/test/cataract/image_287.png   cataract         0.9999   

   prob_normal  
0       0.0000  
1       0.0064  
2       0.0000  
3       0.0000  
4       0.0001  


In [64]:
df_results['cat'] = df_results['prob_cataract']>0.50

In [65]:
df_results['cat'].sum()

57

In [66]:
df_results = infer_on_folder("processed_images/test/normal/", clf, preprocess, model)

🔍 Running Inference:  25%|▎| 1libpng warning: iCCP: known incorrect sRGB profile
🔍 Running Inference:  43%|▍| 2libpng warning: iCCP: known incorrect sRGB profile
🔍 Running Inference:  62%|▌| 3libpng warning: iCCP: known incorrect sRGB profile
🔍 Running Inference:  72%|▋| 4libpng warning: iCCP: known incorrect sRGB profile
🔍 Running Inference:  90%|▉| 5libpng warning: iCCP: known incorrect sRGB profile
🔍 Running Inference: 100%|█| 6


In [67]:
df_results['cat'] = df_results['prob_normal']>0.50

In [68]:
df_results['cat'].sum()

60

## Check effect of augmentation

In [70]:
import torch
import open_clip
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from glob import glob
from torchvision import transforms
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from lightgbm import LGBMClassifier
import warnings
import os

warnings.filterwarnings('ignore')

# === 1. Load CLIP Model ===
def load_clip(device='cuda' if torch.cuda.is_available() else 'cpu'):
    model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
    model.to(device).eval()
    tokenizer = open_clip.get_tokenizer('ViT-B-32')
    return model, preprocess, tokenizer, device

# === 2. Pupil Cropping ===
def crop_to_pupil(image_path, output_size=(784, 784)):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.medianBlur(gray, 5)

    circles = cv2.HoughCircles(gray_blur, cv2.HOUGH_GRADIENT, dp=1.5, minDist=30,
                                param1=50, param2=30, minRadius=20, maxRadius=150)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        x, y, r = circles[0][0]
        pad = int(r * 1.5)
        x1, y1 = max(0, x - pad), max(0, y - pad)
        x2, y2 = min(image.shape[1], x + pad), min(image.shape[0], y + pad)
        cropped = image[y1:y2, x1:x2]
    else:
        print(f"⚠️ Pupil not detected in {image_path}, using full image.")
        cropped = image

    resized = cv2.resize(cropped, output_size)
    return resized

# === 3. Save Cropped Images ===
def preprocess_folder(input_folder, output_folder, size=(784, 784)):
    os.makedirs(output_folder, exist_ok=True)
    for fname in tqdm(os.listdir(input_folder)):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            path = os.path.join(input_folder, fname)
            cropped = crop_to_pupil(path, output_size=size)
            save_path = os.path.join(output_folder, fname)
            cv2.imwrite(save_path, cropped)

# === 4. Augmentation (for training only) ===
augmentation_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.2)
])

def augment_image(img_pil, n_augmentations=2):
    augmented = [augmentation_transforms(img_pil) for _ in range(n_augmentations)]
    grayscale = transforms.Grayscale()(img_pil)
    augmented.append(grayscale.convert("RGB"))

    np_img = np.array(img_pil)
    gray = cv2.cvtColor(np_img, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    edge_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    edge_pil = Image.fromarray(edge_rgb)
    augmented.append(edge_pil)
    return augmented

# === 5. Build DataFrame for Training ===
def build_dataframe(folder, label, preprocess, model, device='cuda', augment=True, n_aug=3):
    df = pd.DataFrame(columns=range(512))
    idx = 0
    for image_path in tqdm(glob(f"{folder}/*.png")):
        image = Image.open(image_path).convert("RGB")
        images = [image] + (augment_image(image, n_augmentations=n_aug) if augment else [])

        for img in images:
            with torch.no_grad(), torch.cuda.amp.autocast():
                tensor = preprocess(img).unsqueeze(0).to(device)
                feat = model.encode_image(tensor)
                feat = feat / feat.norm(dim=-1, keepdim=True)
                df.loc[idx, list(range(512))] = feat.cpu().numpy()[0]
                df.loc[idx, 'category'] = label
                idx += 1
    return df

# === 6. Train Classifier ===
def train_classifier(X, y):
    model = LGBMClassifier(random_state=42)
    model.fit(X, y)
    return model

# === 7. Evaluate Model ===
def evaluate_model(model, X_val, y_val):
    y_pred = model.predict(X_val)
    print(confusion_matrix(y_val, y_pred))
    print(classification_report(y_val, y_pred))

# === 8. Inference with Flags ===
def infer_on_folder(folder_path, clf, preprocess, model, device='cuda',
                    crop=True, strict_preprocess=True, batch_size=16):
    results = []
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for i in tqdm(range(0, len(image_paths), batch_size), desc="🔍 Batched Inference"):
        batch_paths = image_paths[i:i + batch_size]
        images = []

        for path in batch_paths:
            if crop:
                img_cv = crop_to_pupil(path)
                img = Image.fromarray(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))
            else:
                img = Image.open(path).convert("RGB")

            if strict_preprocess:
                img = preprocess(img)
            else:
                img = transforms.Resize((224, 224))(img)
                img = transforms.ToTensor()(img)

            images.append(img)

        with torch.no_grad(), torch.cuda.amp.autocast():
            batch = torch.stack(images).to(device)
            embeddings = model.encode_image(batch)
            embeddings = embeddings / embeddings.norm(dim=-1, keepdim=True)
            embeddings = embeddings.cpu().numpy()

        preds = clf.predict(embeddings)
        probs = clf.predict_proba(embeddings)

        for j, path in enumerate(batch_paths):
            results.append({
                'image_path': path,
                'prediction': 'cataract' if preds[j] == 1 else 'normal',
                'prob_cataract': round(probs[j][1], 4),
                'prob_normal': round(probs[j][0], 4),
            })

    return pd.DataFrame(results)

# === 9. Main Execution ===



In [72]:
import time

# Load model and preprocessor
model, preprocess, tokenizer, device = load_clip()

# Step A: Preprocess raw eye images (optional if already done)
preprocess_folder("processed_images/train/normal", "processed_aug_aug/train/normal")
preprocess_folder("processed_images/train/cataract", "processed_aug_aug/train/cataract")

# Step B: Build dataset with augmentation
start_train_time = time.time()
df_normal = build_dataframe("processed_aug_aug/train/normal", 0, preprocess, model, device=device)
df_cataract = build_dataframe("processed_aug_aug/train/cataract", 1, preprocess, model, device=device)
df = pd.concat([df_normal, df_cataract]).astype(float).sample(frac=1).reset_index(drop=True)

X = df.iloc[:, :512]
y = df['category']

# Step C: Train model
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)
clf = train_classifier(X_train, y_train)
evaluate_model(clf, X_val, y_val)

# Step D: (Optional) Grid search
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
}
grid = GridSearchCV(LGBMClassifier(verbose=-1), param_grid, cv=3)
grid.fit(X_train, y_train)
print("Best model:", grid.best_estimator_)
print(grid.best_params_)

end_train_time = time.time()
print(f"\n⏱️ Total training time (embedding + training): {end_train_time - start_train_time:.2f} seconds")

# Step E: Inference on cataract test images WITH strict_preprocess
start_infer_strict = time.time()
df_results_cat_strict = infer_on_folder(
    "processed_images/test/cataract/", clf, preprocess, model,
    device=device, crop=True, strict_preprocess=True, batch_size=32
)
end_infer_strict = time.time()
print(df_results_cat_strict.head())
print("Predicted cataract count (strict):", (df_results_cat_strict['prob_cataract'] > 0.5).sum())
print(f"⏱️ Inference time with strict_preprocess=True: {end_infer_strict - start_infer_strict:.2f} seconds")

# Step F: Inference on cataract test images WITHOUT strict_preprocess
start_infer_fast = time.time()
df_results_cat_fast = infer_on_folder(
    "processed_images/test/cataract/", clf, preprocess, model,
    device=device, crop=True, strict_preprocess=False, batch_size=32
)
end_infer_fast = time.time()
print(df_results_cat_fast.head())
print("Predicted cataract count (fast):", (df_results_cat_fast['prob_cataract'] > 0.5).sum())
print(f"⏱️ Inference time with strict_preprocess=False: {end_infer_fast - start_infer_fast:.2f} seconds")

# (Optional) Repeat above block for normal images if needed


  3%| | 7/246 [00:00<00:28,  8.libpng warning: iCCP: known incorrect sRGB profile
 15%|▏| 37/246 [00:15<00:46,  4libpng warning: iCCP: known incorrect sRGB profile
 16%|▏| 40/246 [00:16<00:48,  4libpng warning: iCCP: known incorrect sRGB profile
 22%|▏| 53/246 [00:24<01:12,  2libpng warning: iCCP: known incorrect sRGB profile
 30%|▎| 74/246 [00:42<02:52,  1libpng warning: iCCP: known incorrect sRGB profile
 75%|▊| 185/246 [01:00<00:04, 1

⚠️ Pupil not detected in processed_images/train/normal/image_34.png, using full image.


 78%|▊| 192/246 [01:06<00:21,  libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
 83%|▊| 205/246 [01:14<00:20,  libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
 85%|▊| 209/246 [01:17<00:20,  libpng warning: iCCP: known incorrect sRGB profile
 91%|▉| 225/246 [01:39<00:29,  libpng warning: iCCP: known incorrect sRGB profile
 97%|▉| 238/246 [01:43<00:04,  libpng warning: iCCP: known incorrect sRGB profile
100%|█| 246/246 [01:47<00:00,  
  5%| | 12/245 [00:06<01:58,  1libpng warning: iCCP: known incorrect sRGB profile
 10%| | 25/245 [00:10<00:34,  6libpng warning: iCCP: known incorrect sRGB profile
 13%|▏| 32/245 [00:13<01:44,  2libpng warning: iCCP: known incorrect sRGB profile
 24%|▏| 60/245 [00:25<01:26,  2libpng warning: iCCP: known incorrect sRGB profile
 29%|▎| 71/245 [00:31<01:42,  1libpng warning: iCCP: known incorrect sRGB profile
 29%|▎| 72/245 [00:31<01:40,  1libpng warning:

[[276  20]
 [ 15 279]]
              precision    recall  f1-score   support

         0.0       0.95      0.93      0.94       296
         1.0       0.93      0.95      0.94       294

    accuracy                           0.94       590
   macro avg       0.94      0.94      0.94       590
weighted avg       0.94      0.94      0.94       590

Best model: LGBMClassifier(max_depth=7, n_estimators=200, verbose=-1)
{'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200}

⏱️ Total training time (embedding + training): 147.10 seconds


🔍 Batched Inference:  50%|▌| 1libpng warning: iCCP: known incorrect sRGB profile
🔍 Batched Inference: 100%|█| 2


                                     image_path prediction  prob_cataract  \
0  processed_images/test/cataract/image_279.png   cataract         0.9997   
1  processed_images/test/cataract/image_251.png   cataract         0.9992   
2  processed_images/test/cataract/image_292.png   cataract         0.9997   
3  processed_images/test/cataract/image_286.png   cataract         0.9995   
4  processed_images/test/cataract/image_287.png   cataract         0.9977   

   prob_normal  
0       0.0003  
1       0.0008  
2       0.0003  
3       0.0005  
4       0.0023  
Predicted cataract count (strict): 56
⏱️ Inference time with strict_preprocess=True: 40.40 seconds


🔍 Batched Inference:  50%|▌| 1libpng warning: iCCP: known incorrect sRGB profile
🔍 Batched Inference: 100%|█| 2

                                     image_path prediction  prob_cataract  \
0  processed_images/test/cataract/image_279.png   cataract         0.8214   
1  processed_images/test/cataract/image_251.png   cataract         0.9812   
2  processed_images/test/cataract/image_292.png   cataract         0.9991   
3  processed_images/test/cataract/image_286.png   cataract         0.9958   
4  processed_images/test/cataract/image_287.png   cataract         0.9989   

   prob_normal  
0       0.1786  
1       0.0188  
2       0.0009  
3       0.0042  
4       0.0011  
Predicted cataract count (fast): 54
⏱️ Inference time with strict_preprocess=False: 39.37 seconds


In [89]:
import random
random.choice(['Head',"Tail"])

'Head'